***
# PRESSURE-VELOCITY COUPLING
***

* To aid the following discussion, we present the steady 3-D incompressible equations for mass and momentum conservation.
   
    * Mass conservation:
    
        $$\frac{\partial u_x}{\partial x} + \frac{\partial u_y}{\partial y} + \frac{\partial u_z}{\partial z} = 0 \tag{1}$$
    
    * Momentum Conservation:
    
        $$\rho\left[u_x\frac{\partial u_x}{\partial x} + u_y\frac{\partial u_x}{\partial y} + u_z\frac{\partial u_x}{\partial z}\right] = -\frac{\partial p}{\partial x} + \mu \left(\frac{\partial ^2 u_x}{\partial x^2} + \frac{\partial ^2 u_x}{\partial y^2} + \frac{\partial ^2 u_x}{\partial z^2}\right) \tag{2}$$

        $$\rho\left[u_x\frac{\partial u_y}{\partial x} + u_y\frac{\partial u_y}{\partial y} + u_z\frac{\partial u_y}{\partial z}\right] = -\frac{\partial p}{\partial y} + \mu \left(\frac{\partial ^2 u_y}{\partial x^2} + \frac{\partial ^2 u_y}{\partial y^2} + \frac{\partial ^2 u_y}{\partial z^2}\right) \tag{3}$$

        $$\rho\left[u_x\frac{\partial u_z}{\partial x} + u_y\frac{\partial u_z}{\partial y} + u_z\frac{\partial u_z}{\partial z}\right] = -\frac{\partial p}{\partial z} + \mu \left(\frac{\partial ^2 u_z}{\partial x^2} + \frac{\partial ^2 u_z}{\partial y^2} + \frac{\partial ^2 u_z}{\partial z^2}\right)\tag{4}$$
    
*  Having discussed various discretization schemes, at this point it may seem trivial solving the above set of equations. And while the momentum equations are non-linear due to the dependence of advection coefficients $\rho u$ on the dependent variable $u$, which we are trying to solve for, the main challenge is the pressure term. Nonlinearity can easily be handled by employing an iterative solution procedure. On the other hand, both pressure and velocity fields are unknown a priori.

* One may ask why pressure cannot be specified through an equation of state via the density which can be solved for through the continuity equation or better yet eliminate it entirely through the stream function-vorticity formulation, which was presented in one of the notebooks. The former is only possible for compressible flows and does not work for incompressible flows. There are a few concerns about adopting the latter approach:
    
    1. It is not always desirable to eliminate pressure since it is an important variable. There are ways to calculate pressure from solution obtained via the vorticity method, but this requires significant computational effort which may offset the gains realized from eliminating pressure in the first place.
    
    2. Vorticity is difficult to specify at a wall and an arbitrary value can cause convergence issues.
    
    3. The main concern is that the vorticity method cannot be extended to 3-D problems for which a stream function does not exist. For 3-D problems, vorticity formulation results in 6 dependent variables, including 3 vorticity components and 3 velocity components. This is in contrast to the pressure-velocity formulation which has 4 dependent variables only (3 velocity components, 1 pressure variable). 

        $$\omega = \nabla \times \mathbf{u} = \text{det} \begin{vmatrix}\frac{\partial}{\partial x} && \frac{\partial}{\partial y} && \frac{\partial}{\partial z}\\ u_x && u_y && u_z\end{vmatrix} = \begin{bmatrix}\left(\frac{\partial u_y}{\partial x} - \frac{\partial u_x}{\partial y}\right) && \left(\frac{\partial u_z}{\partial x} - \frac{\partial u_x}{\partial z}\right) && \left(\frac{\partial u_z}{\partial y} - \frac{\partial u_y}{\partial z}\right) \end{bmatrix} \tag{5}$$

* Since the stream function-vorticity method is undesirable, we need to couple the continuity and momentum equations somehow. One difficulty with this exercise is how to treat the first derivatives of pressure and velocity. As shown elsewhere, the central difference scheme was found to be sufficient for second derivatives that show up in the diffusion term. However, use of central difference for the first derivatives found in the advection term was shown to be oblivious to the state of node P. Alternative schemes that recognize the unidirectional nature of advection were proposed. Although not made explicit, the velocity components were specified at the cell face, while the advected property $\phi$ was specified at the nodes, with their cell face values determined through interpolation. A similar treatment can be employed here in specifying the velocity components and pressure variable on a grid. This type of specification where the pressure is located at the centers of the control volumes and velocities at the boundaries of the control volumes is often referred to as a staggered grid. This specification does require a computer program that can keep track of geometric locations of the velocities and perform interpolations.

## VELOCITY CORRECTION EQUATIONS
***

* A staggered grid requires a different control volume definition for the momentum equations, for which $\phi$ is the velocity component. The new control volume should be centered on the location of the velocity components, with pressure specified at its boundaries, rather than at the center.

* The discretized momentum equation takes the following form, where the subscript **nb** represents neigbours, **b** is the discretized source term and **A** is the cell face area. The sum of neighbour terms includes advection and diffusion terms.

    $$a_e u_{x,e} = \sum a_{nb} u_{x,nb} + b + \left(P_P - P_E\right) A_e \tag{6}$$
    
    $$a_w u_{x,w} = \sum a_{nb} u_{x,nb} + b + \left(P_W - P_P\right) A_w \tag{7}$$

    $$a_n u_{y,n} = \sum a_{nb} u_{y,nb} + b + \left(P_P - P_N\right) A_n \tag{8}$$
    
    $$a_s u_{y,s} = \sum a_{nb} u_{y,nb} + b + \left(P_S - P_P\right) A_s \tag{9}$$
    
    $$a_t u_{z,t} = \sum a_{nb} u_{z,nb} + b + \left(P_P - P_T\right) A_t \tag{10}$$
    
    $$a_b u_{z,b} = \sum a_{nb} u_{z,nb} + b + \left(P_B - P_P\right) A_b \tag{11}$$
    
* Both pressure and velocity are unknown in the above equations. An initial guess of the pressure field $P^*$ can be made to solve for an intermediate velocity $u^*$. The velocity obtained this way is denoted with an asterisk because it is based on a guessed pressure field, which may not satisfy continuity.

* Substituting the guessed pressure field and the intermediate velocity solution into the original equations:

    $$a_e u_{x,e}^* = \sum a_{nb} u_{x,nb}^* + b + \left(P_P^* - P_E^*\right) A_e \tag{12}$$
    
    $$a_w u_{x,w}^* = \sum a_{nb} u_{x,nb}^* + b + \left(P_W^* - P_P^*\right) A_w \tag{13}$$
    
    $$a_n u_{y,n}^* = \sum a_{nb} u_{y,nb}^* + b + \left(P_P^* - P_N^*\right) A_n \tag{14}$$
    
    $$a_s u_{y,s}^* = \sum a_{nb} u_{y,nb}^* + b + \left(P_S^* - P_P^*\right) A_s \tag{15}$$
    
    $$a_t u_{z,t}^* = \sum a_{nb} u_{z,nb}^* + b + \left(P_P^* - P_T^*\right) A_t \tag{16}$$
    
    $$a_b u_{z,b}^* = \sum a_{nb} u_{z,nb}^* + b + \left(P_B^* - P_P^*\right) A_b \tag{17}$$
    
* To improve our guess of the pressure field so that the resulting velocity solution is closer to satisfying the continuity, we apply a correction $P'$ to the guessed pressure $P^*$ such that the "correct" pressure $P$ is given by:
    
     $$P = P^* + P' \tag{18}$$
     
* Such a correction to the guessed pressure field is considered to result in a correction to the velocity $u'$, such that the "correct" velocity solution $u$ is given by:
    
    $$u_x = u_x^* + u_x'\tag{19}$$
    
    $$u_y = u_y^* + u_y'\tag{20}$$
    
    $$u_z = u_z^* + u_z'\tag{21}$$
    
* Substracting equations 12-17 from equations 6-11.

    $$a_e \left(u_{x,e} - u_{x,e}^*\right) = \sum a_{nb} \left(u_{x,nb} - u_{x,nb}^*\right) + \left[\left(P_P - P_P^*\right) - \left(P_E - P_E^*\right)\right] A_e\tag{22}$$
    
    $$a_w \left(u_{x,w} - u_{x,w}^*\right) = \sum a_{nb} \left(u_{x,nb} - u_{x,nb}^*\right) + \left[\left(P_W - P_W^*\right) - \left(P_P - P_P^*\right)\right] A_w \tag{23}$$
    
    $$a_n \left(u_{y,n} - u_{y,n}^*\right) = \sum a_{nb} \left(u_{y,nb} - u_{y,nb}^*\right) + \left[\left(P_P - P_P^*\right) - \left(P_N - P_N^*\right)\right] A_n \tag{24}$$
    
    $$a_s \left(u_{y,s} - u_{y,s}^*\right) = \sum a_{nb} \left(u_{y,nb} - u_{y,nb}^*\right) + \left[\left(P_S - P_S^*\right) - \left(P_P - P_P^*\right)\right] A_s \tag{25}$$
    
    $$a_t \left(u_{z,t} - u_{z,t}^*\right) = \sum a_{nb} \left(u_{z,nb} - u_{z,nb}^*\right)  + \left[\left(P_P - P_P^*\right) - \left(P_T - P_T^*\right)\right] A_t \tag{26}$$
    
    $$a_b \left(u_{z,b} - u_{z,b}^*\right) = \sum a_{nb} \left(u_{z,nb} - u_{z,nb}^*\right)  + \left[\left(P_B - P_B^*\right) - \left(P_P - P_P^*\right)\right] A_b \tag{27}$$
    
* The above equations can be simplified to:
    
    $$a_e u'_{x,e}  = \sum a_{nb} u'_{x,nb} + \left[P'_P - P'_E\right] A_e \tag{28}$$
    
    $$a_w u'_{x,w}  = \sum a_{nb} u'_{x,nb} + \left[P'_W - P'_P\right] A_w \tag{29}$$
    
    $$a_n u'_{y,n} = \sum a_{nb} u'_{y,nb} + \left[P'_P - P'_N\right] A_n \tag{30}$$
    
    $$a_s u'_{y,s} = \sum a_{nb} u'_{y,nb} + \left[P'_S - P'_P\right] A_s \tag{31}$$
    
    $$a_t u'_{z,t} = \sum a_{nb} u'_{z,nb} + \left[P'_P - P'_T\right] A_t \tag{32}$$
    
    $$a_b u'_{z,b} = \sum a_{nb} u'_{z,nb} + \left[P'_B - P'_P\right] A_b \tag{33}$$
    
* For simplicity, we shall drop the neighbouring terms. It will become apparent later when deriving the pressure correction equation why this ommision has no influence on the final converged solution.

    $$u'_{x,e} = u_{x,e} - u_{x,e}^* = \left[P'_P - P'_E\right] \frac{A_e}{a_e}\tag{34}$$
    
    $$u'_{x,w} = u_{x,w} - u_{x,w}^* = \left[P'_W - P'_P\right] \frac{A_w}{a_w} \tag{35}$$
    
    $$u'_{y,n} = u_{y,n} - u_{y,n}^* = \left[P'_P - P'_N\right] \frac{A_n}{a_n} \tag{36}$$
    
    $$u'_{y,s} = u_{y,s} - u_{y,s}^* = \left[P'_S - P'_P\right] \frac{A_s}{a_s} \tag{37}$$
    
    $$u'_{z,t} = u_{z,t} - u_{z,t}^* = \left[P'_P - P'_T\right] \frac{A_t}{a_t} \tag{38}$$
    
    $$u'_{z,b} = u_{z,b} - u_{z,b}^* = \left[P'_B - P'_P\right] \frac{A_b}{a_b} \tag{39}$$
    
* Re-arranging above equations, we get the following velocity correction equations:

    $$u_{x,e} = u_{x,e}^* + \left[P'_P - P'_E\right] d_e \tag{40}$$
    
    $$u_{x,w} = u_{x,w}^* + \left[P'_W - P'_P\right] d_w \tag{41}$$
    
    $$ u_{y,n} =  u_{y,n}^* + \left[P'_P - P'_N\right] d_n \tag{42}$$
    
    $$ u_{y,s} =  u_{y,s}^* + \left[P'_S - P'_P\right] d_s\tag{43}$$
    
    $$u_{z,t} = u_{z,t}^* + \left[P'_P - P'_T\right] d_t\tag{44}$$
    
    $$u_{z,b} = u_{z,b}^* + \left[P'_B - P'_P\right] d_b\tag{45}$$

## PRESSURE CORRECTION EQUATION
***

* The continuity equation is discretized as follows:
    
    $$\left[\left(\rho u_x\right)_e - \left(\rho u_x\right)_w\right]\Delta y \Delta z + \left[\left(\rho u_y\right)_n - \left(\rho u_y\right)_s\right]\Delta x \Delta z + \left[\left(\rho u_z\right)_t - \left(\rho u_z\right)_b\right]\Delta x \Delta y = 0 \tag{46}$$
    
* Replacing the velocity components in the above equations by the velocity correction equations. Due to the incompressible nature of the problem, $\rho$ is taken to be constant.

    $$\rho \Delta y \Delta z\left\{\left[u_{x,e}^* + d_e\left(P'_P - P'_E\right)\right] - \left[u_{x,w}^* + d_w\left(P'_W - P'_P\right)\right]\right\} + \rho \Delta x \Delta z\left\{\left[u_{y,n}^* + d_n\left(P'_P - P'_N\right)\right] - \left[u_{y,s}^* + d_s\left(P'_S - P'_P\right)\right]\right\} + \rho \Delta x \Delta y\left\{\left[u_{z,t}^* + d_t\left(P'_P - P'_T\right)\right] - \left[u_{z,b}^* + d_b\left(P'_B - P'_P\right)\right]\right\} = 0 \tag{47}$$
    
* Casting the above equation into the usual form, we get the following pressure correction equation:

    $$a_P P'_P = a_E P'_E + a_W P'_W + a_S P'_S + a_N P'_N + a_B P'_B + a_T P'_T + b \tag{48}$$
    
    $$a_E = \rho d_e \Delta y \Delta z \tag{49}$$
    
    $$a_W = \rho d_w \Delta y \Delta z \tag{50}$$
    
    $$a_S = \rho d_s \Delta x \Delta z \tag{51}$$
    
    $$a_N = \rho d_n \Delta x \Delta z \tag{52}$$
    
    $$a_B = \rho d_b \Delta x \Delta y \tag{53}$$
    
    $$a_T = \rho d_t \Delta x \Delta y \tag{54}$$
    
    $$a_P = a_E + a_W + a_S + a_N + a_B + a_T \tag{55}$$
    
    $$b = \rho \left[\left(u_{x,w}^* - u_{x,e}^*\right)\Delta y \Delta z + \left(u_{y,s}^* - u_{y,n}^*\right)\Delta x \Delta z + \left(u_{z,b}^* - u_{z,t}^*\right)\Delta x \Delta y\right] \tag{56}$$
    
* The coefficient $\textit{b}$ shown above is the negative of the discretized incompressible continuity equation shown in equation 46. As such, if the value of $\textit{b}$ is zero, then the intermediate velocities $u^*$ obtained from a guessed pressure field $P^*$ satisfy continuity and no pressure correction is required. The coefficient $\textit{b}$ also happens to be the sum of neighbouring terms that was dropped while deriving the velocity correction equations. Dropping $\textit{b}$ in the first place does not influence the final solution since $\textit{b}$ becomes zero when the solution has converged. However, it does affect the path to convergence and often equation 18 needs to be under-relaxed to prevent the solution from diverging. 

    $$P = P^* + \alpha P' \tag{57}$$

## SEMI-IMPLICIT METHOD FOR PRESSURE-LINKED EQUATIONS (SIMPLE)
***

* This solution procedure was developed by Patankar and Spalding. It proceeds as follows:
    
    1. Starting from a guessed pressure field, $P^*$, compute the intermediate velocities $u^*$ by solving equations 6-11.
    
    2. Compute pressure corrections $P'$ by substituting intermediate velocities $u^*$ into equation 48.
    
    3. Calculate new pressure field $P$ from $P^*$ and $P'$ using equation 18
    
    4. Calculate new velocity field $u$ using the velocity correction equations (40-45)
    
    5. Treat the new pressure field obtained in step 3 as the new guessed pressure field and repeat the procedure until the solution has converged.
    
* Due to slow convergence and even divergence in some cases, a few improvements on the SIMPLE algorithm have been developed. Some of these solution procedures include SIMPLER, SIMPLEC, PIMPLE and PISO. The reader is invited to consult relevant literature for more information.